In [3]:
# Load all necessary packages for AIF360
import sys
import os
sys.path.insert(1, "../")  

import numpy as np
np.random.seed(0)

from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

In [15]:
import domino

project = "{}/{}".format(os.environ['DOMINO_PROJECT_OWNER'],
                         os.environ['DOMINO_PROJECT_NAME'])

print("Project name: {}".format(project))

dmm_model_id = "644ffac7a9872366aac7065d"
d = domino.Domino(project)
metrics_client = d.custom_metrics_client()

Project name: dave_heinicke/AI-Fairness-360


In [16]:
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [22]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

original_mean_difference = abs(metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


In [23]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

corrected_mean_difference = abs(metric_orig_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


### Log as a Domino Model Monitoring Custom metric

In [29]:

# metrics_client.log_metric(dmm_model_id, "Age_Mean_Difference", age_mean_difference, "2023-5-08T00:00:00Z", { "example_tag1" : "value1", "example_tag2" : "value2" })

metrics_client.log_metrics([
{ "modelMonitoringId" : dmm_model_id, "metric" : "Age_Mean_Difference", "value" : corrected_mean_difference,
"timestamp" : "2023-05-08T00:00:00Z",
"tags" : { "example_tag1" : "value1", "example_tag2" : "value2" }
},
{ "modelMonitoringId" : dmm_model_id, "metric" : "Age_Mean_Difference", "value" : original_mean_difference,
"timestamp" : "2023-05-08T00:00:10Z" }
])

Set the custom trigger alert for model monitoring

In [36]:
metrics_client.trigger_alert(dmm_model_id, "Age_Mean_Difference", 3.14,
                            condition = metrics_client.GREATER_THAN,
                            lower_limit=0.1,
                            upper_limit=Mo
                            description = "Age Difference Factor Greater than 0.1 detected" )

ApiTypeError: Invalid type. Required value type is Decimal and passed type was NoneClass at ['args[0]']['upperLimit']

In [32]:
res = metrics_client.read_metrics(dmm_model_id, "Age_Mean_Difference",
"2023-05-08T00:00:00Z", "2023-05-08T00:00:10Z")

print(res)

{'metadata': {'requestId': '790befb1-fd51-494c-8ea3-2fafc2f09057', 'notices': ()}, 'metricValues': [{'timestamp': '2023-05-08T00:00:00Z', 'value': -0.1699054740619017, 'tags': {'example_tag1': 'value1', 'example_tag2': 'value2'}}, {'timestamp': '2023-05-08T00:00:10Z', 'value': -0.1699054740619017, 'tags': {}}]}
